In [1]:
import pandas as pd

### Import all bridge projects in Allegheny County

In [2]:
future = pd.read_excel('input/future-development.xlsx')
indev = pd.read_excel('input/in-development.xlsx')
construct = pd.read_excel('input/under-construction.xlsx')

/Users/jon/.local/share/virtualenvs/bridger-lo3S_JL1/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/jon/.local/share/virtualenvs/bridger-lo3S_JL1/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/jon/.local/share/virtualenvs/bridger-lo3S_JL1/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
df = pd.concat([future, indev, construct])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2071 entries, 0 to 925
Columns: 127 entries, GPID to COMPLETED
dtypes: float64(17), int64(30), object(80)
memory usage: 2.0+ MB


### Restrict to Pittsburgh-owned bridges, only include projects once

In [5]:
df = df[df['Project Sponsor Name'] == 'City of Pittsburgh']

In [6]:
df['Project Stage'] = df['Project Stage'].str.title()
df['PUBLIC_NARRATIVE'] = df['PUBLIC_NARRATIVE'].str.replace('(Sponor = City of Pittsburgh) ', '', regex=False)
df['PUBLIC_NARRATIVE'] = df['PUBLIC_NARRATIVE'].str.replace('(Sponsor = City of Pittsburgh)', '', regex=False)
df['PUBLIC_NARRATIVE'] = df['PUBLIC_NARRATIVE'].str.replace('(Project sponsor = City of Pittsburgh)', '', regex=False)

In [7]:
df = df.drop_duplicates('Project ID')
df = df.reset_index(drop=True)

In [8]:
df.loc[df['Completion Date'].notna(), 'Completion Date DT'] = pd.to_datetime(df['Completion Date'], format='%Y%m%d')

df.loc[df['Completion Date'].isna(), 'COMPLETION_DATE'] = 'Unknown'
df.loc[df['Completion Date'].notna(), 'COMPLETION_DATE'] = df['Completion Date DT'].dt.strftime('%m/%d/%Y')

In [9]:
df['NTP Date'] = pd.to_datetime(df['NTP Date'], format='%Y%m%d')

df['NTP_DATE'] = ''
df.loc[df['NTP Date'].notna(), 'NTP_DATE'] = df['NTP Date'].dt.strftime('%m/%d/%Y')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Columns: 130 entries, GPID to NTP_DATE
dtypes: datetime64[ns](2), float64(16), int64(30), object(82)
memory usage: 24.5+ KB


### Properly format

In [11]:
df[[
    'PUBLIC_NARRATIVE',
    'Project Stage',
    'Estimated CONS Phase Cost Amount',
    'NTP_DATE',
    'COMPLETION_DATE'
]]

,PUBLIC_NARRATIVE,Project Stage,Estimated CONS Phase Cost Amount,NTP_DATE,COMPLETION_DATE
0,Superstructure replacement on Ansonia Place ov...,Future Development,550000.0,07/14/2031,Unknown
1,Bridge preservation on South Millvale Avenue b...,Future Development,15380000.0,08/14/2034,Unknown
2,Bridge rehabilitation on Lowrie Street over Ri...,Future Development,750000.0,06/21/2032,Unknown
3,Bridge preservation on Mission Street between ...,Future Development,8984000.0,06/14/2032,Unknown
4,"Bridge preservation on SR 400, Baum Boulevard ...",Future Development,1500000.0,06/14/2032,10/15/2034
5,Tunnel reconstruction and preservation work o...,Future Development,17510000.0,09/11/2034,12/01/2035
6,Bridge rehabilitation on structure carrying S...,In Development,5005000.0,09/13/2023,01/21/2025
7,"Bridge rehabilitation on Swinburne Bridge, wh...",In Development,12300000.0,03/09/2026,11/04/2025
8,Bridge restoration/replacement on Larimer Ave...,In Development,21000000.0,08/26/2025,06/30/2027
9,Swindell Bridge Rehabilitation located between...,In Development,30000000.0,12/07/2026,01/07/2028


In [13]:
df[[
    'PUBLIC_NARRATIVE',
    'Project Stage',
    'Estimated CONS Phase Cost Amount',
    'NTP_DATE',
    'COMPLETION_DATE'
]].to_csv('output/pgh-bridges.csv', index=False)